In [1]:
import pandas as pd

In [ ]:
species = pd.read_csv("/media/muskrat/T7 Shield/eco_data/v4/db_exports/species.csv")

In [ ]:
species

In [ ]:
native = pd.read_parquet(
    "/media/muskrat/T7 Shield/eco_data/v4/combined/native_checklisted_v2.parquet"
)

native

In [3]:
checklist = pd.read_parquet("")

In [ ]:
duplicates_1 = pd.read_parquet("")

duplicates_2 = pd.read_parquet("")

In [ ]:
checklist

In [5]:
# rename canonicalName to scientific_name in checklist

checklist = checklist.rename(columns={"canonicalName": "scientific_name"})

In [8]:
# rename acceptedName to scientific_name in species

species = species.rename(columns={"acceptedName": "scientific_name"})

In [ ]:
species_small = species[["scientific_name", "unique_id"]]

species_small

In [ ]:
merge = pd.merge(species_small, checklist, on="scientific_name", how="left")

merge

In [ ]:
# find duplicates in merge scientific_name column

duplicates = merge[merge.duplicated(["scientific_name"], keep=False)]

duplicates

In [10]:
duplicates.to_parquet("")

In [ ]:
# remove all values in merge that are in duplicates

merge = merge[~merge.scientific_name.isin(duplicates_1.scientific_name)]

merge = merge[~merge.scientific_name.isin(duplicates_2.scientific_name)]

merge

In [ ]:
# check for duplicates in merge

duplicates = merge[merge.duplicated(["scientific_name"], keep=False)]

duplicates

In [12]:
del duplicates_1, duplicates_2, duplicates

In [ ]:
# accepted_small is only merge columns scientific_name, unique_id, and acceptedName

accepted_small = merge[["scientific_name", "unique_id", "acceptedName"]]

accepted_small

In [ ]:
bufo_small = accepted_small[accepted_small["acceptedName"] == "Anaxyrus americanus"]

bufo_small

In [ ]:
for i in range(len(accepted_small)):
    accepted_small["unique_id"].iloc[i] = (
        accepted_small["unique_id"]
        .iloc[i]
        .replace('["', "")
        .replace('"]', "")
        .split('","')
    )


accepted_small

In [ ]:
# group scientific_name by acceptedName

names = (
    accepted_small.groupby(["acceptedName"])["scientific_name"]
    .apply(list)
    .reset_index()
)

names

In [ ]:
# find rows where acceptedName is Anaxyrus americanus

bufo_name = names[names["acceptedName"] == "Antaresia childreni"]

bufo_name

In [ ]:
ids = accepted_small.groupby(["acceptedName"])["unique_id"].apply(list).reset_index()

ids

In [ ]:
for i in range(len(ids)):
    print(i)
    cat = []
    for j in range(len(ids["unique_id"].iloc[i])):
        for k in ids["unique_id"].iloc[i][j]:
            # print(k)
            if k not in cat:
                cat.append(k)

    ids["unique_id"].iloc[i] = cat

ids

In [ ]:
bufo_id = ids[ids["acceptedName"] == "Antaresia childreni"]

bufo_id

In [ ]:
test_1 = ["47", "50", "52"]

test_2 = [
    "50",
    "51",
    "52",
    "53",
    "55",
    "57",
    "63",
    "67",
    "71",
    "72",
    "73",
    "74",
    "75",
    "76",
    "77",
    "79",
    "80",
    "81",
    "82",
]

# create list of unique values from test_1 and test_2

test_3 = list(set(test_1 + test_2))

test_3

In [ ]:
# check if test 3 is the same as bufo_3


bufo_3 = [
    "47",
    "50",
    "52",
    "51",
    "53",
    "55",
    "57",
    "63",
    "67",
    "71",
    "72",
    "73",
    "74",
    "75",
    "76",
    "77",
    "79",
    "80",
    "81",
    "82",
]


set(test_3) == set(bufo_3)

In [ ]:
# find difference between test_3 and bufo_3

set(test_3) - set(bufo_3)

In [ ]:
# merge names and ids on acceptedName
species_checked = pd.merge(names, ids, on="acceptedName", how="inner")

species_checked

In [ ]:
# check for duplicates in species_checked

species_checked[species_checked.duplicated(subset=["acceptedName"])]

In [ ]:
species_checked.to_parquet("")

In [ ]:
checklist_small = checklist[
    [
        "acceptedName",
        "taxonRank",
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
    ]
]

checklist_small

In [ ]:
del (
    checklist,
    merge,
    names,
    ids,
    bufo_small,
    bufo_name,
    bufo_id,
    bufo_3,
    test_1,
    test_2,
    test_3,
)

In [20]:
native_checked = species_checked

In [ ]:
del species_checked

In [ ]:
species_checked = pd.read_parquet("")

In [21]:
native_checked

In [ ]:
species_checked

In [22]:
# drop column scientific_name from species_checked and native_checked

species_checked = species_checked.drop(["scientific_name"], axis=1)

native_checked = native_checked.drop(["scientific_name"], axis=1)

In [ ]:
# rename unique_id column in native_checked to native_ecoregions

native_checked = native_checked.rename(columns={"unique_id": "native_ecoregions"})

native_checked

In [ ]:
# merge native_checked and species_checked

all_species = pd.merge(native_checked, species_checked, on="acceptedName", how="outer")

all_species

In [ ]:
# find rows where native_ecoregions is not NaN

all_species[~all_species.native_ecoregions.isnull()]

In [ ]:
# WE CURRENTLY SET UNIQUE_ID TO NATIVE_ECOREGIONS IF NO UNIQUE IDS ARE PRESENT BUT NATIVE ECOREGIONS ARE PRESENT

# if unique_id is NaN in all species, set unique_id to native_ecoregions

all_species.loc[all_species.unique_id.isnull(), "unique_id"] = (
    all_species.native_ecoregions
)

all_species

In [ ]:
no_unique = pd.merge(native_checked, species_checked, on="acceptedName", how="left")

no_unique

In [ ]:
# find rows where unique_id is NaN

no_unique = no_unique[no_unique.unique_id.isnull()]

no_unique

In [ ]:
# merge no_unique and checklist_small

no_unique_checklist = pd.merge(
    no_unique, checklist_small, on="acceptedName", how="left"
)

no_unique_checklist

In [ ]:
# remove duplicates in no_unique_checklist

no_unique_checklist = no_unique_checklist[
    ~no_unique_checklist.duplicated(["acceptedName"], keep="first")
]

no_unique_checklist

In [ ]:
# find rows where taxonRank is variety

no_unique_checklist[no_unique_checklist.taxonRank == "variety"]

In [ ]:
# find duplicates in no_unique_checklist

duplicates = all_species_checklist[
    all_species_checklist.duplicated(["acceptedName"], keep=False)
]

duplicates

In [ ]:
no_unique_checklist.to_parquet("")

In [ ]:
all_species_checklist = pd.merge(
    all_species, checklist_small, on="acceptedName", how="left"
)

all_species_checklist

In [ ]:
all_species_checklist = all_species_checklist[
    ~all_species_checklist.duplicated(["acceptedName"], keep="first")
]

all_species_checklist

In [ ]:
# find unique values of taxonRank in all_species_checklist

all_species_checklist.taxonRank.unique()

In [ ]:
# show rows where taxonRank is not species

all_species_checklist[all_species_checklist.taxonRank != "species"]

In [ ]:
# drop column taxonRank from all_species_checklist

all_species_checklist = all_species_checklist.drop(["taxonRank"], axis=1)

# rename column acceptedName to scientific_name

all_species_checklist = all_species_checklist.rename(
    columns={"acceptedName": "scientific_name"}
)

all_species_checklist

In [ ]:
# drop column native_ecoregions_y from all_species_checklist and rename column native_ecoregions_x to native_ecoregions in all_species_checklist

all_species_checklist = all_species_checklist.drop(["native_ecoregions_y"], axis=1)

all_species_checklist = all_species_checklist.rename(
    columns={"native_ecoregions_x": "native_ecoregions"}
)

all_species_checklist

In [35]:
all_species_checklist.to_parquet(
    "/media/muskrat/T7 Shield/eco_data/v4/combined/native_checklisted_v2.parquet"
)

In [ ]:
del all_species, checklist_small, duplicates, species_checked

In [ ]:
species_type = species[["scientific_name", "species_type", "common_name"]]

species_type

In [ ]:
# merge all_species_checklist and species_type

final = pd.merge(all_species_checklist, species_type, on="scientific_name", how="left")

final

In [ ]:
# rename column database to native_rights in native

native = native.rename(columns={"database": "native_rights"})

native

In [ ]:
# drop unique_id column from native

native = native.drop(["unique_id"], axis=1)

native

In [ ]:
native_common = native[["scientific_name", "common_name"]]

native_common

In [ ]:
# drop common_name from native

native = native.drop(["common_name"], axis=1)

native

In [ ]:
# merge final and native on scientific_name

final = pd.merge(final, native, on="scientific_name", how="left")

final

In [ ]:
final_common = final[["scientific_name", "common_name"]]

final_common

In [ ]:
# merge final_common and native_common on scientific_name and common_name

common = pd.merge(
    final_common, native_common, on=["scientific_name", "common_name"], how="left"
)

common

In [ ]:
# drop common_name from final and merge with common on scientific_name

final = final.drop(["common_name"], axis=1)

final = pd.merge(final, common, on="scientific_name", how="left")

final

In [ ]:
final.to_parquet("")

In [ ]:
# rename unique_id column to observed_ecoregions in final

final = final.rename(columns={"unique_id": "observed_ecoregions"})

final

In [ ]:
final.to_parquet("")